# Homework 4b Submission
## Yonadav Shavit

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.special import factorial
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

(Let's define some latex commands)
 
$\newcommand{\floor}[1]{\left \lfloor #1 \right \rfloor}$
$\newcommand{\clamp}[1]{\left [ #1 \right ]}$
$\newcommand{\lap}[1]{Lap\left ( #1 \right )}$
$\newcommand{\lapcdf}[2]{LapCDF\left ( #1, [-\inf, #2] \right )}$
$\newcommand{\lappdf}[2]{LapPDF\left ( #1, |_{#2} \right )}$
$\newcommand{\lapcdffull}[2]{\frac{1}{2} + \frac{1}{2}sgn\left({#2}\right)\left ( 1 - \exp{\left ( - \frac{|#2|}{#1} \right ) }\right )}$
$\newcommand{\lappdffull}[2]{\frac{1}{2 #1} \exp{\left ( - \frac{#2}{#1}\right ) }}$

## 1.
### (a)
#### i.
$GS_f = \infty$ because the sensitivity is unbounded and so a single point can arbitrarily change the mean.
#### ii.
$\min _{x \in \mathcal{S}} \operatorname{LS}_{f}(x) = \infty$ for the same reasons as above - regardless of any of the other points (i.e. $x$ s.t. we get minimum local sensitivity), a single additional point can arbitrarily change the resulting mean. (For example, to increase the mean by $k$, the new point should be $\mu_{old} + nk$.)
#### iii.
$\mathrm{RS}_{f}^{\mathcal{H}} = \frac{b-a}{n}$ because a single point can change only by $b-a$ while remaining in $H$, and so the average can only change by that divided by $n$.
#### iv.
We can construct an explicit Lipschitz extension by simply clipping each point to $[a, b]$ and then taking the mean and adding Laplace noise with sensitivity $\frac{b-a}{n\epsilon}$. This will be increasingly biased as the original data is farther from $[a, b]$, but the sensitivity will remain the same, and on $H$ this agrees with $f$.

### (b)
#### i.
$GS_f = \infty$ because, for any large $N$, we can construct a dataset with $\frac{n}{2}$ data points at $0$, and $\frac{n}{2}$ datapoints at $N$, and then shifting a single point from $0$ to $N$ shifts the median by $N$, for any arbitrary $N$.
#### ii.
$\min _{x \in \mathcal{S}} \operatorname{LS}_{f}(x) = 0$ because if we create a dataset with $3$ points at $0$, no matter how much we change any single point, the other two will still be $0$ and so the median will be unchanged.
#### iii.
$\mathrm{RS}_{f}^{\mathcal{H}} = b-a$ because every point must fall in $[a, b]$, and so the largest possible change in median would be from $a$ to $b$. (This example is possible if half the points minus 1 are at $a$, and the other half are at $b$, and then we move one from $b$ to $a$).

### (c)
#### i.
$GS_f = n-1$ because if every node was previously disconnected (meaning the value was $n-1$) we can connect a new node to every old node, meaning that now $0$ nodes are disconnected.
#### ii.
$\min _{x \in \mathcal{S}} \operatorname{LS}_{f}(x) = 1$ because if every node in the graph is already connected, the addition of a new node cannot disconnect any existing nodes. The most it can do is add a node with no edges. (Note that our definition of node sensitivity in this case is a little weird, because if we're talking about deleting nodes rather than adding, the minimum local sensitivity is actually $0$, if all edges exist between all nodes.)
#### iii.
$\mathrm{RS}_{f}^{\mathcal{H}} = d$ because the greatest change from adding a single node would be that all existing nodes are unconnected, and then a single new node is added that connects to $d$ other nodes. 
(It can't connect to more than that because it must be in $H$). 
Thus the query can only decrease by $d$.

## 2.

Below is the R code, reproduced. I'm porting it into Python.

In [13]:
## Here is the likelihood function for a Logit
# calcllik<-function(b,data){           
#   y<-data[,1]
#   x<-data[,2]

#   pi<- 1/(1+exp(-b[1] - b[2]*x))        # Here is the systematic component

#   llik<-y * log(pi) + (1-y) * log(1-pi) # Here is the stocastic component
#   return(-llik)
# }

def likelihood(w, b, x, y):
    pi = 1/(1 + np.exp(-b - w*x.reshape(-1)))
    llik = y*np.log(pi + 1e-9) + (1-y)*np.log(1e-9 + 1-pi)
    print(f"likelihood var: {llik.std()}")
    return - llik

In [14]:
## Differentially private mean release

# gaussianReleaseNoise <- function(size=1, sensitivity, epsilon, delta){
# 	scale <- sensitivity *log(1.25/delta)/ epsilon
# 	noise <- rnorm(n=size, mean=0, sd=scale)
# 	return(noise)
# }

def gaussianReleaseNoise(sensitivity, epsilon, delta, size=1):
    scale = sensitivity*np.log(1.25/delta)/epsilon
    noise = np.random.normal(scale=scale, size=size)
    return noise

In [15]:
#### Run with actual data

# library("foreign")
# PUMSdata <- read.csv(file="../../data/MaPUMS5full.csv")

# mydata<-PUMSdata[c("married","educ")]

# output <- glm(married ~ educ, family="binomial", data=mydata)

# print(summary(output))

PUMSdata = pd.read_csv("../data/MaPUMS5full.csv")
X = (PUMSdata['educ'].values).reshape(-1, 1)
Y = PUMSdata['married'].values
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=10000).fit(X, Y)

print(f"slope = {lr.coef_[0]}, bias = {lr.intercept_}")

FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.

> /Users/yonadavshavit/anaconda3/envs/py36/lib/python3.7/site-packages/sklearn/linear_model/logistic.py(433)_check_solver()
    431         warnings.warn("Default solver will be changed to 'lbfgs' in 0.22. "
    432                       "Specify a solver to silence this warning.",
--> 433                       FutureWarning)
    434 
    435     all_solvers = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']

ipdb> quit


In [16]:
# #### Show the estimated model

# xseq <- seq(from=-40, to=60, length=100)
# f <- 1/(1 + exp(-output$coef[1] -output$coef[2]*xseq))

# par(mfcol=c(2,1))

# plot(xseq, f, type="l", lwd=1.5, col="red", ylim=c(0,1), ylab="E(y|x,theta)", xlab="education", main="Probability Married by Education")
# abline(v=1, col="blue", lty=3)
# abline(v=16, col="blue", lty=3)

# plot(xseq, f, type="l", lwd=1.5, col="red", ylim=c(0,1), ylab="E(y|x,theta)", xlab="education", xlim=c(-5,20))
# for(i in 1:16){
# 	flag<-mydata$educ==i
# 	points(x=i, y=mean(mydata[flag,"married"]))
# }
# dev.copy2pdf(file="./figs/married_educ.pdf")


# #### Show the LogLikelihood surface

# sample.data <- mydata[sample(1:nrow(mydata),10000), ]
# b1.seq <- seq(from=-3, to=2, length=25)
# b2.seq <- seq(from=-.2, to=.3, length=25)
# llsurface <- matrix(NA, nrow=length(b1.seq), ncol=length(b2.seq))

# for(i in 1:length(b1.seq)){
# 	for(j in 1:length(b2.seq)){
# 		llsurface[i,j] <- sum(-1* calcllik(b=c(b1.seq[i], b2.seq[j]), data=sample.data) )
# 	}
# }

# filled.contour(x=b1.seq, y=b2.seq, z=llsurface, color = terrain.colors,  xlab="constant parameter", ylab="education parameter")

# dev.copy2pdf(file="./figs/logitLLike.pdf")

In [17]:
# calcgradient <- function(B, C, theta, fun){
# 	dx <- 0.0001

# 	out1 <-	eval(fun(b=theta, data=B))
# 	out2 <- eval(fun(b=theta + c(0,dx), data=B))
# 	out3 <- eval(fun(b=theta + c(dx,0), data=B))

# 	Del.1 <- 1
# 	# Del.1 <- clip(Del.1, lower=0, upper=1)  # Fix this
# 	mean.Del.1 <- mean(Del.1)

# 	Del.2 <- 1
# 	# Del.2 <- clip(Del.2, lower=0, upper=1)  # Fix this
# 	mean.Del.2 <- mean(Del.2)

# 	return(c(mean.Del.1,mean.Del.2))
# }

def calc_clipped_gradient(X, Y, C, w, b, fun=likelihood):
    dx = 1e-4
    out1 = fun(w     ,      b, X, Y)
    out2 = fun(w     , b + dx, X, Y)
    out3 = fun(w + dx, b     , X, Y)
    dw = (out3 - out1)/dx
    db = (out2 - out1)/dx
    scaling_param = np.minimum(1, C/np.sqrt(dw**2 + db**2))
    return dw*scaling_param, db*scaling_param

In [18]:
# N <- nrow(mydata)
# L <- round(sqrt(nrow(mydata)))

# steps <- 10   	  # Fix this

# ## Shuffle the data
# index <- sample(1:nrow(mydata))
# mydata <- mydata[index,]
# epsilon <-1

# theta <- c(0,0)   # Starting parameters
# C <- 10			  # Interval to clip over
# nu <- c(1,0.01)   # Learning speeds


# history <- matrix(NA, nrow=steps+1, ncol=2)
# history[1,] <- theta

# for(i in 1:steps){
# 	startB <- ((i-1)*L+1)
# 	if(i<L){
# 		stopB <- i*L
# 	}else{
# 		stopB <- nrow(mydata)
# 	}

# 	index<-sample(1:nrow(mydata),L)
# 	B <- mydata[startB:stopB, ]
# 	Del <- calcgradient(B, C, theta, fun=calcllik)
# 	cat("Del:  ",Del,"\n")
# 	theta <- theta   				# Fix this
# 	cat("Theta:",theta, "\n")

# 	history[i+1,] <- theta

# }
def learn_private_regression(epsilon, X_, Y_, delta=1e-6):
    n = Y_.shape[0]
    batchsize = int(np.sqrt(n))
    steps = int(np.sqrt(n))

    idxs = np.random.permutation(n)
    X = X_[idxs, :]
    Y = Y_[idxs]

    w = 0; b = 0         # starting parameters
    C = 10               # clipping parameter
    nu_w, nu_b = .00001, .001  # learning rates

    w_history = [w]
    b_history = [b]
    
    for i in tqdm_notebook(range(steps)):
        print(f"b:{b}")
        batch_start = i*batchsize
        batch_end = min((i + 1)*batchsize, n)
        Xbatch = X[batch_start:batch_end, :]
        Ybatch = Y[batch_start:batch_end]
        dw_clipped, db_clipped = calc_clipped_gradient(Xbatch, Ybatch, C, w, b, fun=likelihood)
        dw_private = dw_clipped + gaussianReleaseNoise(
            sensitivity=2*C, # one sample can shift from -C to C, so double sensitivity
            epsilon=epsilon/(batchsize*steps*2), # two parameters released, so halve the epsilon for each
            delta=delta,
            size=batchsize,
        )
        db_private = db_clipped  + gaussianReleaseNoise(
            sensitivity=2*C/batchsize, # one sample can shift from -C to C, so double sensitivity
            epsilon=(epsilon/(2*steps)), # two parameters released, so halve the epsilon for each
            delta=delta,
            size=batchsize,
        )
        dw_private = dw_private.mean()
        db_private = db_private.mean()
        # update via calculated gradients
        w += nu_w*dw_private
        b += nu_b*db_private
        w_history.append(w)
        b_history.append(b)
        if True:
            print(f"Likelihood: {likelihood(w, b, X, Y).mean()}")
    return w_history, b_history

In [19]:
X, Y = X[:10000, :], Y[:10000]
import warnings
warnings.simplefilter("error")
w_history, b_history = learn_private_regression(1, X, Y)

b:0
likelihood var: 2.7103998191910124e-16
likelihood var: 4.853864430087968e-05
likelihood var: 0.0005318635355073509
likelihood var: 10.000500815710028
Likelihood: 8.23940923357495
b:0.04231599160889713
likelihood var: 10.024761260755643
likelihood var: 10.02476334213856
likelihood var: 10.024772218600631



RuntimeWarning: divide by zero encountered in true_divide

> <ipython-input-17-be4d921c9216>(26)calc_clipped_gradient()
     23     out3 = fun(w + dx, b     , X, Y)
     24     dw = (out3 - out1)/dx
     25     db = (out2 - out1)/dx
---> 26     scaling_param = np.minimum(1, C/np.sqrt(dw**2 + db**2))
     27     return dw*scaling_param, db*scaling_param

ipdb> quit


In [ ]:
par(mfcol=c(2,1))

all.ylim<-c( min(c(history[,1],output$coef[1] )), max(c(history[,1],output$coef[1] )))
plot(history[,1], type="l", ylim=all.ylim, ylab="beta 0", xlab="step", lwd=1.5)
abline(h=output$coef[1], lty=2, col="blue", lwd=1.5)


all.ylim<-c( min(c(history[,2],output$coef[2] )), max(c(history[,2],output$coef[2] )))
plot(history[,2], type="l", ylim=all.ylim, ylab="beta 1", xlab="step", lwd=1.5)
abline(h=output$coef[2], lty=2, col="blue", lwd=1.5)

dev.copy2pdf(file="./figs/dpSGD.pdf")